In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [28]:
#import necessary libraries
import os

import torch
from torchvision import datasets, transforms
from torch.utils.data import Subset, DataLoader
import torch.nn as nn
import timm
import torch.optim as optim

from sklearn.model_selection import train_test_split
from collections import Counter

from PIL import Image

In [29]:
#Key Settings
DataDir = "/content/drive/MyDrive/Colab Notebooks/FabricDefectDetection/Dataset"
BatchSize = 16
ImageSize = 224
TrainRatio = 0.8
RandomSeed = 42
torch.manual_seed(RandomSeed)

In [30]:
#To count images per class before splitting
print("Images before splitting across class: \n")
classes = sorted(os.listdir(DataDir))
for cls in classes:
  classpath = os.path.join(DataDir,cls)
  count = len(os.listdir(classpath))
  print(f"{cls}:{count}")



Images before splitting across class: 

Broken stitch:112
Needle mark:108
Pinched fabric:108
Vertical:101
defect free:1666
hole:281
horizontal:136
lines:157
stain:398


In [38]:
#Load Dataset and apply transformations
train_transform = transforms.Compose( [
    transforms.Resize((ImageSize, ImageSize)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness = 0.2, contrast = 0.2),
    transforms.ToTensor()
]

)

val_transform = transforms.Compose( [
    transforms.Resize((ImageSize, ImageSize)),
    transforms.ToTensor()

]
)

FabricImages = datasets.ImageFolder(DataDir,transform = val_transform)

In [39]:
# Stratified Split

# Representing position of all images in the dataset
indices = list(range(len(FabricImages)))

#To get the labels
labels = [FabricImages.imgs[i][1] for i in indices]

#to perform stratified split
train_id, val_id = train_test_split(
    indices,
    test_size = 0.2,
    stratify = labels,
    random_state = 42
)



In [40]:
# To create train and validataion datasets
#To take a subset of the original dataset (train and val) and apply specific transformation to it
#Requirement: Training data with Augmentation and Validation data without Augmentation
#Additonal: from torch.utils.data import Subset; its purpose is to select a subset of a dataset using list of indices

class SubsetwithTransform(Subset):
  def __init__(self,subset,transform):
    # subset - a PyTorch subset object containing subset.dataset and subset.indices
    super().__init__(subset.dataset, subset.indices)
    self.transform = transform

  def __getitem__(self, idx):
    img,label = super().__getitem__(idx)
    if self.transform:
      img = self.transform(img)
    return img, label


train_dataset = SubsetwithTransform(Subset(FabricImages,train_id),train_transform)
val_dataset = SubsetwithTransform(Subset(FabricImages,val_id),val_transform)


In [41]:
#Create DataLoaders
train_loader = DataLoader(train_dataset,batch_size = BatchSize, shuffle = True)
val_loader = DataLoader(val_dataset,batch_size = BatchSize, shuffle = False)

In [42]:
#Define The Model
num_classes = len(FabricImages.classes)
model = timm.create_model('vit_base_patch16_224',pretrained = True)
model.head = nn.Linear(model.head.in_features, num_classes )

#Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

#weight = class_weights; The loss will now penalize misclassification of minority classes more, helping the model learn them better

#Specify the loss
counts = [Counter(labels)[i] for i in range(num_classes)]
total = sum(counts)
weights = [total/c for c in counts]
class_weights = torch.tensor(weights,dtype=torch.float).to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights)

#specify the optimizer
optimizer = optim.Adam(model.parameters(),lr=3e-4)



In [43]:
#Training
num_epochs = 10

for epoch in range(num_epochs):
  #put the model in training mode:

  model.train()

  running_loss = 0.0 # to compute batch loss
  correct = 0
  total = 0

  for images,labels in train_loader:
    images,labels = images.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item() *images.size(0)#To get the total loss of the batch by multiplying average loss of the batch with batch size
    _,predicted_class = outputs.max(1)
    total += labels.size(0) #to track of how many instances processed so far
    correct += predicted_class.eq(labels).sum().item()

  train_loss = running_loss / total
  train_acc = (100 * correct) / total

  #validation
  model.eval()
  val_loss = 0.0
  correct = 0
  total = 0

  with torch.no_grad():
    for images,labels in val_loader:
      images,labels = images.to(device),labels.to(device)
      outputs = model(images)
      loss = criterion(outputs, labels)
      val_loss += loss.item() * images.size(0)
      _,predicted_class = outputs.max(1)
      total +=labels.size(0)
      correct += predicted_class.eq(labels).sum().item()

  val_loss = val_loss / total
  val_accuracy = (100*correct) / total

  print(f"Epoch:[{epoch+1}/{num_epochs}] |"
        f"Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.2f}% |",
        f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}% "
        )


torch.save(model.state_dict(), "vit_fabric_defect.pth")




Epoch:[1/10] |Training Loss: 1.8155, Training Accuracy: 42.23% | Validation Loss: 1.5118, Validation Accuracy: 42.83% 
Epoch:[2/10] |Training Loss: 1.2472, Training Accuracy: 56.79% | Validation Loss: 0.9397, Validation Accuracy: 70.36% 
Epoch:[3/10] |Training Loss: 1.0449, Training Accuracy: 70.40% | Validation Loss: 0.8932, Validation Accuracy: 66.29% 
Epoch:[4/10] |Training Loss: 0.9783, Training Accuracy: 72.85% | Validation Loss: 0.9824, Validation Accuracy: 63.68% 
Epoch:[5/10] |Training Loss: 0.9671, Training Accuracy: 72.16% | Validation Loss: 0.9214, Validation Accuracy: 69.22% 
Epoch:[6/10] |Training Loss: 0.7288, Training Accuracy: 79.74% | Validation Loss: 0.6093, Validation Accuracy: 83.88% 
Epoch:[7/10] |Training Loss: 0.7494, Training Accuracy: 77.33% | Validation Loss: 0.9823, Validation Accuracy: 68.89% 
Epoch:[8/10] |Training Loss: 0.7505, Training Accuracy: 79.13% | Validation Loss: 0.6090, Validation Accuracy: 83.22% 
Epoch:[9/10] |Training Loss: 0.7247, Training Ac